In [14]:
import json
from tqdm import tqdm

In [48]:
# Load CUHK validate data and targeted LCSH
etd_json = []
with open('./data/cuhk_etd_abstract_validate.json','r') as f:
    for line in f.readlines():
        line = line.strip("\n")
        if not line:
            continue
        etd_json.append(json.loads(line))
        
etd_result = []
# with open('./data/cuhk_pretrained_wv_result.json','r') as f:
with open('./result/no_cuhk_etd_as_training_pretrained_wv_maxout/cuhk_etd_abstract_validate_prediction.json', 'r') as f:
    for line in f.readlines():
        line = line.strip("\n")
        if not line:
            continue
        etd_result.append(json.loads(line))
        
with open('./data/enough_train_data_lcsh.json','r') as f:
    enough_lcsh = json.load(f)

In [16]:
len(enough_lcsh)

2928

### Calculate f1-measure (>= 25 only)

In [8]:
targeted_lcsh = enough_lcsh.keys()

In [9]:
TP = {i:0.0 for i in targeted_lcsh}
TN = {i:0.0 for i in targeted_lcsh}
FP = {i:0.0 for i in targeted_lcsh}
FN = {i:0.0 for i in targeted_lcsh}

In [10]:
for i in tqdm(targeted_lcsh):
    for true,pred in zip(etd_json,etd_result):
        t = true['lcsh'].keys()
        p = pred.keys()
        if i in t and i in p:
            TP[i] += 1
        if i not in t and i not in p:
            TN[i] += 1
        if i not in t and i in p:
            FP[i] += 1
        if i in t and i not in p:
            FN[i] += 1

100%|██████████| 2928/2928 [00:21<00:00, 136.06it/s]


In [11]:
f_measure = {i:{} for i in targeted_lcsh}

In [12]:
for i in tqdm(targeted_lcsh):
    f_measure[i]['precision'] = TP[i] / (TP[i] + FP[i] + 1e-13)
    f_measure[i]['recall'] = TP[i] / (TP[i] + FN[i] + 1e-13)
    f_measure[i]['f1'] = 2*f_measure[i]['precision']*f_measure[i]['recall'] / (f_measure[i]['precision']+f_measure[i]['recall']+1e-13)

100%|██████████| 2928/2928 [00:00<00:00, 629306.80it/s]


In [13]:
macro_precision = sum(f_measure[i]['precision'] for i in targeted_lcsh)/len(targeted_lcsh)
macro_recall = sum(f_measure[i]['recall'] for i in targeted_lcsh)/len(targeted_lcsh)
macro_f1 = 2*macro_precision*macro_recall/(macro_precision+macro_recall+1e-13)

In [14]:
macro_precision,macro_recall,macro_f1

(0.12668266475820214, 0.1454432223591526, 0.13541626028048218)

### Calculate hit@5 >= 25 only

In [15]:
hit_5 = 0.0
cnt = 0
for true,pred in tqdm(zip(etd_json,etd_result)):
    matched = 0
    t = true['lcsh'].keys()
    p = pred.keys()
    if any(elem in t for elem in targeted_lcsh):
        for i in t:
            if i in p:
                matched += 1
        cnt += 1
    hit_5 += matched/len(t)
hit_5 = hit_5 / cnt

10373it [00:01, 8725.11it/s]


In [16]:
hit_5

0.1892589316149126

### Calculate f1-measure

In [49]:
targeted_lcsh = []
for i in etd_json:
    targeted_lcsh += list(i['lcsh'].keys())
targeted_lcsh = set(targeted_lcsh)

In [18]:
TP = {i:0.0 for i in targeted_lcsh}
TN = {i:0.0 for i in targeted_lcsh}
FP = {i:0.0 for i in targeted_lcsh}
FN = {i:0.0 for i in targeted_lcsh}

In [19]:
for i in tqdm(targeted_lcsh):
    for true,pred in zip(etd_json,etd_result):
        t = true['lcsh'].keys()
        p = pred.keys()
        if i in t and i in p:
            TP[i] += 1
        if i not in t and i not in p:
            TN[i] += 1
        if i not in t and i in p:
            FP[i] += 1
        if i in t and i not in p:
            FN[i] += 1

100%|██████████| 6927/6927 [00:49<00:00, 141.31it/s]


In [20]:
f_measure = {i:{} for i in targeted_lcsh}

In [21]:
for i in tqdm(targeted_lcsh):
    f_measure[i]['precision'] = TP[i] / (TP[i] + FP[i] + 1e-13)
    f_measure[i]['recall'] = TP[i] / (TP[i] + FN[i] + 1e-13)
    f_measure[i]['f1'] = 2*f_measure[i]['precision']*f_measure[i]['recall'] / (f_measure[i]['precision']+f_measure[i]['recall']+1e-13)

100%|██████████| 6927/6927 [00:00<00:00, 713733.36it/s]


In [22]:
macro_precision = sum(f_measure[i]['precision'] for i in targeted_lcsh)/len(targeted_lcsh)
macro_recall = sum(f_measure[i]['recall'] for i in targeted_lcsh)/len(targeted_lcsh)
macro_f1 = 2*macro_precision*macro_recall/(macro_precision+macro_recall+1e-13)

In [23]:
macro_precision,macro_recall,macro_f1

(0.06502004446284776, 0.06971948733087048, 0.06728781235661276)

### Calculate hit@5

In [56]:
hit_5 = 0.0
cnt = 0
for true,pred in tqdm(zip(etd_json,etd_result)):
    matched = 0
    t = true['lcsh'].keys()
#     p = pred.keys()
    p = [i for i,_ in sorted(pred.items(), key=lambda x:x[1], reverse=True)[:5]]
#     if any(elem in t for elem in targeted_lcsh):
    for i in t:
        if i in p:
            matched += 1
    cnt += 1
    hit_5 += matched/len(t)
hit_5 = hit_5 / cnt

10373it [00:00, 126069.28it/s]


In [57]:
hit_5

0.128152185389725

In [35]:
idx = 30
etd_json[idx]['lcsh'],[i for i,_ in sorted(etd_result[idx].items(), key=lambda x:x[1], reverse=True)[:5]]

({'Computer graphics': 1, 'Lighting': 1},
 ['Computer science',
  'Electrical engineering',
  'Image processing',
  'Optics',
  'Mechanical engineering'])

In [12]:
[i for i,_ in sorted(etd_result[0].items(), key=lambda x:x[1], reverse=True)[:5]]

['Personnel management', 'Business']

In [8]:
etd_result[0]

{'Personnel management': 0.5543419718742371,
 'Business': 0.5442637801170349,
 'Management': 0.5279812216758728,
 'International business enterprises': 0.4131326377391815,
 'Industrial management': 0.38191887736320496}